In [4]:
import os
import glob
import pandas as pd
import csv

In [5]:
# if data dir in hpcc
os.chdir("/bigdata/statsdept/xfeng044/stat209/xfeng044/data")

#  Use glob to match the pattern ‘csv’
dir1 = 'Summary_Details/202*/202*.'
dir2 = 'Summary_Hashtag/202*/202*.'
dir3 = 'Summary_Sentiment/202*/202*.'
file_type = 'csv'
all_summ = [i for i in glob.glob(dir1 + file_type)]
all_hash = [i for i in glob.glob(dir2 + file_type)]
all_sent = [i for i in glob.glob(dir3 + file_type)]
# print(all_summ[1:5])

In [34]:
####-----------------------FOR Hashtag--------------#######
# Combine all files from a given start_time and end_time

## define start date
startTime = '2021-01-01 00:00:00'
endTime = '2021-02-28 23:00:00'

## define the file scope
files = all_hash
# files = ['2020_05_31_23_Summary_Hashtag.csv',
#  '2019_05_31_23_Summary_Hashtag.csv',
#  '2020_01_31_23_Summary_Hashtag.csv',
#  '2020_03_30_22_Summary_Hashtag.csv']


## define start and end time
startTime = pd.to_datetime(startTime)
endTime = pd.to_datetime(endTime)


## conbine file in certain interval
out = []
for x in files:
    temp = '_'.join(x.split('/')[2].split('_',4)[:-1])                    
    date = pd.to_datetime(temp,  format='%Y_%m_%d_%H')

    if date >= startTime and date <= endTime:
        temp = pd.read_csv(x,encoding= 'unicode_escape')
        #print(date)
        temp['Year'] = date.year
        temp['Month'] = date.month
        temp['Day'] = date.day
        out.append(temp)
        
combined_csv = pd.concat(out, ignore_index=True)
# # test output
# combined_csv.head(5)


In [35]:
##------get top x hashtags per day-------##
combined_csv = combined_csv.groupby(['Year','Month','Day','Hastag']).count() ##### 1-9 use hashtag, 10-12 use hastag in original table
combined_csv = combined_csv.sort_values(['Year','Month','Day','Tweet_ID'],ascending=False).groupby(['Month','Day']).head(100).reset_index()

combined_csv.to_csv('combined_hash_from_{}_to_{}.csv'.format(startTime.date(),endTime.date()), 
                     index=False,encoding="utf-8",errors='ignore')

In [150]:
def combine_hash(start,end,types):
    files = types

    ## define start and end time
    startTime = pd.to_datetime(start)
    endTime = pd.to_datetime(end)

    ## conbine file in certain interval
    out = []
    for x in files:
        temp = '_'.join(x.split('/')[2].split('_',4)[:-1])                    
        date = pd.to_datetime(temp,  format='%Y_%m_%d_%H')
        #print(date)
        if date >= startTime and date <= endTime:
            temp = pd.read_csv(x,encoding= 'unicode_escape')
            temp['Year'] = date.year
            temp['Month'] = date.month
            temp['Day'] = date.day
            out.append(temp)

    combined_csv = pd.concat(out, ignore_index=True)
    return combined_csv

##-------------test combine_hash-------##
combine_hash('2020-01-22 00:00:00','2020-01-31 23:00:00', all_hash)

,Tweet_ID,Hashtag,Year,Month,Day
0,1223335138370760705,#China,2020,1,31
1,1223335138370760705,#coronavirus,2020,1,31
2,1223335138370760705,#CoronavirusOutbreak,2020,1,31
3,1223335138756648965,#coronaravirus,2020,1,31
4,1223335138895056899,#coronavirus,2020,1,31
...,...,...,...,...,...
5153518,1220602122007281671,#coronavirus,2020,1,24
5153519,1220602122007281671,#CoronavirusOutbreak,2020,1,24
5153520,1220602124725153793,#coronavirus,2020,1,24
5153521,1220602126587420677,#CoronavirusOutbreak,2020,1,24


In [6]:
####-----------------------FOR Summary Detail & Geo code data--------------#######

## define the file scope
files = all_summ

## define start date
startTime = '2021-04-01 00:00:00'
endTime = '2021-04-30 23:00:00'

## define start and end time
startTime = pd.to_datetime(startTime)
endTime = pd.to_datetime(endTime)

## define file_name
summ = 'Sum_combine_'+startTime.strftime("%Y%m%d")+'-'+endTime.strftime("%Y%m%d")
with_geo = 'Geo_combine_'+startTime.strftime("%Y%m%d")+'-'+endTime.strftime("%Y%m%d")

## conbine file in certain interval
summ = []
with_geo = pd.DataFrame()

for x in files:
    temp = '_'.join(x.split('/')[2].split('_',4)[:-1])                    
    date = pd.to_datetime(temp,  format='%Y_%m_%d_%H')
    #print(date)
    if date >= startTime and date <= endTime:
        temp = pd.read_csv(x,encoding= 'unicode_escape')
        temp_count = temp.shape[0]
        temp_engcount = temp[temp['Language']=='en'].shape[0]
        temp_geocount = temp[temp['Country'].notnull()].shape[0]

        summ.append([date.year,date.month,date.day,date.hour,temp_count,temp_engcount,temp_geocount])
        
        temp_geo = temp[temp['Country'].notnull()][['Tweet_ID','Language','Geolocation_coordinate','Country','Date Created']]
        with_geo = with_geo.append(temp_geo,ignore_index=True)

summ = pd.DataFrame(summ,columns=['Year','Month','Day','Hour','Count','Eng_count','Geo_count']) 

#export combined file to csv
summ.to_csv('combined_summ_from_{}_to_{}.csv'.format(startTime.date(),endTime.date()), 
                    index=False, encoding="utf-8",errors='ignore')

with_geo.to_csv('combined_geo_from_{}_to_{}.csv'.format(startTime.date(),endTime.date()), 
                    index=False, encoding="utf-8",errors='ignore')

In [8]:
####-----------------------FOR SENTIMENT SUMM--------------#######

## define the file scope
files = all_sent

## define start date
startTime = '2021-04-01 00:00:00'
endTime = '2021-04-30 23:00:00'

## define start and end time
startTime = pd.to_datetime(startTime)
endTime = pd.to_datetime(endTime)

## define file_name
sent = 'Sent_combine_'+startTime.strftime("%Y%m%d")+'-'+endTime.strftime("%Y%m%d")

## conbine file in certain interval
sent = []


for x in files:
    temp = '_'.join(x.split('/')[2].split('_',4)[:-1])                    
    date = pd.to_datetime(temp,  format='%Y_%m_%d_%H')
    #print(date)
    if date >= startTime and date <= endTime:
        temp = pd.read_csv(x,encoding= 'unicode_escape')
        neg_count = temp[temp['Sentiment_Label']=='negative'].shape[0]
        pos_count = temp[temp['Sentiment_Label']=='positive'].shape[0]
        neu_count = temp[temp['Sentiment_Label']=='neutral'].shape[0]
        sent.append([date.year,date.month,date.day,date.hour,neg_count,pos_count,neu_count])
        

sent = pd.DataFrame(sent,columns=['Year','Month','Day','Hour','Neg_Count','Pos_count','Neu_count']) 

# export combined file to csv
sent.to_csv('combined_sent_from_{}_to_{}.csv'.format(startTime.date(),endTime.date()), 
                    index=False, encoding="utf-8",errors='ignore')


In [6]:
#-------------------check memory usage----------------#
combined_csv.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46899740 entries, 0 to 46899739
Data columns (total 5 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   Tweet_ID  int64 
 1   Hashtag   object
 2   Year      int64 
 3   Month     int64 
 4   Day       int64 
dtypes: int64(4), object(1)
memory usage: 4.4 GB
